In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics

from sklearn.decomposition import PCA
from matplotlib.pyplot import figure
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit

import pandas as pd

In [6]:
# Read the CSV file into a DataFrame
file_path = 'urldata.csv'  # Replace with your file path
df = pd.read_csv(file_path)

df = df.drop('Domain', axis=1)

# Separating out a sample for prediction
predict_rows = df.sample(15, random_state=42)
df = df.drop(predict_rows.index)

# Separating features and label
df_Y = df['Label']
df_X = df.drop('Label', axis=1)

# Preparing the training data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_X)
df_X_scaled = scaler.transform(df_X)

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_X_scaled, df_Y, test_size=0.20, random_state=42)

# Transform your predict_rows for prediction

predict_features = predict_rows.drop('Label', axis=1)
predict_labels = predict_rows['Label']

# Scale the features of the 15 rows
predict_features_scaled = scaler.transform(predict_features)


In [7]:
from sklearn.ensemble import StackingClassifier

'''
base_learners = [('rf', RandomForestClassifier(criterion='entropy', max_features='auto', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]
'''

base_learners_set1 = [('rf', RandomForestClassifier(criterion='entropy', max_features='sqrt', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance'))]

base_learners_set2 = [('rf', RandomForestClassifier(criterion='entropy', max_features='sqrt', min_samples_leaf=1, min_samples_split=3, n_estimators=100)), 
                ('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set3 = [('rf', RandomForestClassifier(criterion='entropy', max_features='sqrt', min_samples_leaf=1, min_samples_split=3, n_estimators=100)),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]

base_learners_set4 = [('mlp', MLPClassifier(max_iter=500, activation='relu', alpha=0.001, hidden_layer_sizes=(20,), learning_rate='adaptive', solver='adam')),
                ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=15, n_neighbors=20, p=1, weights='distance')), 
                ('svm', SVC(C=10, kernel='rbf', tol=0.001))]


base_learners = [base_learners_set1, base_learners_set2, base_learners_set3, base_learners_set4]

for base_learner_group in base_learners:
    meta_learner = LogisticRegression()
    clf = StackingClassifier(estimators=base_learner_group, final_estimator=meta_learner)

    # Train the model on the full training data
    clf.fit(X_train, y_train)

    # Predict on the test set and evaluate
    test_predictions = clf.predict(X_test)
    print('Test Set Evaluation:')
    print('Accuracy:', accuracy_score(y_test, test_predictions)*100)
    print('F1 Score:', f1_score(y_test, test_predictions)*100)
    print('Recall:', recall_score(y_test, test_predictions)*100)
    print('Precision:', precision_score(y_test, test_predictions)*100)
    print('ROC AUC:', roc_auc_score(y_test, test_predictions)*100)
    print('Confusion Matrix:', confusion_matrix(y_test, test_predictions))
    print('-----------------------------------------\n')

    # Predict on the 15 rows and compare with actual labels
    predict_predictions = clf.predict(predict_features_scaled)
    print('Prediction on 15 Rows:')
    print('Predicted Labels:', predict_predictions)
    print('Actual Labels:', predict_labels.values)
    print('-----------------------------------------\n')

Test Set Evaluation:
Accuracy: 91.20283510527413
F1 Score: 90.48692515779982
Recall: 83.729662077597
Precision: 98.43060323688081
ROC AUC: 91.19816437213183
Confusion Matrix: [[2368   32]
 [ 390 2007]]
-----------------------------------------

Prediction on 15 Rows:
Predicted Labels: [0 1 1 0 1 0 0 0 0 0 0 0 1 0 0]
Actual Labels: [0 1 1 1 1 0 0 0 1 1 0 1 1 0 0]
-----------------------------------------

Test Set Evaluation:
Accuracy: 91.20283510527413
F1 Score: 90.48692515779982
Recall: 83.729662077597
Precision: 98.43060323688081
ROC AUC: 91.19816437213183
Confusion Matrix: [[2368   32]
 [ 390 2007]]
-----------------------------------------

Prediction on 15 Rows:
Predicted Labels: [0 1 1 0 1 0 0 0 0 0 0 0 1 0 0]
Actual Labels: [0 1 1 1 1 0 0 0 1 1 0 1 1 0 0]
-----------------------------------------

Test Set Evaluation:
Accuracy: 91.20283510527413
F1 Score: 90.48692515779982
Recall: 83.729662077597
Precision: 98.43060323688081
ROC AUC: 91.19816437213183
Confusion Matrix: [[2368   